<a href="https://colab.research.google.com/github/miaomiao612/MiniGPT-4/blob/main/finetune_miniGPT4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/miaomiao612/MiniGPT-4.git

Cloning into 'MiniGPT-4'...
remote: Enumerating objects: 458, done.
remote: Counting objects: 100% (259/259), done.
remote: Compressing objects: 100% (136/136), done.
remote: Total 458 (delta 147), reused 126 (delta 123), pack-reused 199
Receiving objects: 100% (458/458), 91.40 MiB | 19.61 MiB/s, done.
Resolving deltas: 100% (161/161), done.


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')


Mounted at /content/drive/


In [ ]:
%cd /content/MiniGPT-4

/content/MiniGPT-4


# **1.2 准备Vicuna权重**

### 1. Vicuna weights:

In [ ]:
!git lfs install

!git clone https://huggingface.co/lmsys/vicuna-7b-delta-v0  # smaller, need 12G gpu memory

Updated git hooks.
Git LFS initialized.
Cloning into 'vicuna-7b-delta-v0'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 32 (delta 2), reused 0 (delta 0), pack-reused 26
Unpacking objects: 100% (32/32), 7.11 KiB | 1.02 MiB/s, done.
Filtering content: 100% (3/3), 4.55 GiB | 30.21 MiB/s, done.
Encountered 1 file(s) that may not have been copied correctly on Windows:
	pytorch_model-00001-of-00002.bin

See: `git lfs help smudge` for more details.


### 2. LLAMA-7B weight:

In [ ]:
!git clone https://huggingface.co/decapoda-research/llama-7b-hf

Cloning into 'llama-7b-hf'...
remote: Enumerating objects: 91, done.
remote: Total 91 (delta 0), reused 0 (delta 0), pack-reused 91
Unpacking objects: 100% (91/91), 22.58 KiB | 1.41 MiB/s, done.
Filtering content: 100% (34/34), 12.55 GiB | 138.99 MiB/s, done.


### 3. 下载兼容library

In [ ]:
!pip install git+https://github.com/lm-sys/FastChat.git@v0.1.10

  Cloning https://github.com/lm-sys/FastChat.git (to revision v0.1.10) to /tmp/pip-req-build-xjj3q6q0
  Running command git clone --filter=blob:none --quiet https://github.com/lm-sys/FastChat.git /tmp/pip-req-build-xjj3q6q0


### 4. 创建最终权重

将llama-7b-hf/tokenizer_config.json 中的"tokenizer_class": “LLaMATokenizer” 改成 “tokenizer_class”: “LlamaTokenizer”

In [ ]:
!python -m fastchat.model.apply_delta --base /content/MiniGPT-4/llama-7b-hf/  --target /content/MiniGPT-4/vicuna_weights/  --delta /content/MiniGPT-4/vicuna-7b-delta-v0/

Loading the base model from /content/MiniGPT-4/llama-7b-hf/
2023-07-11 19:11:05.550933: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Loading checkpoint shards: 100% 33/33 [00:13<00:00,  2.40it/s]
Loading the delta from /content/MiniGPT-4/vicuna-7b-delta-v0/
Loading checkpoint shards: 100% 2/2 [00:40<00:00, 20.05s/it]
You are using the legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565
Applying the delta
Applying delta: 100% 323/323 [00:05<00:00, 56.37it/s]
Saving the target model to /content/MiniGPT-4/vicuna_weights/


修改minigpt4/configs/models/minigpt4.yaml中“llama_model”为vicuna_weights路径（已改）

# 1.3 微调

download pretrained MiniGPT-4 checkpoint:

In [ ]:
!wget https://huggingface.co/wangrongsheng/MiniGPT4-7B/resolve/main/prerained_minigpt4_7b.pth


--2023-07-11 19:13:13--  https://huggingface.co/wangrongsheng/MiniGPT4-7B/resolve/main/prerained_minigpt4_7b.pth
Resolving huggingface.co (huggingface.co)... 13.224.249.44, 13.224.249.119, 13.224.249.43, ...
Connecting to huggingface.co (huggingface.co)|13.224.249.44|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/25/fa/25fa880e116eda3d82c5c87b32eb0a7fa8f76b139d70ce756851215ce7a76179/017a9ed588a11ed383711003cf50cf675191420a04689f682fb56fa9bbb8dcbb?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27prerained_minigpt4_7b.pth%3B+filename%3D%22prerained_minigpt4_7b.pth%22%3B&Expires=1689361993&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTY4OTM2MTk5M319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy8yNS9mYS8yNWZhODgwZTExNmVkYTNkODJjNWM4N2IzMmViMGE3ZmE4Zjc2YjEzOWQ3MGNlNzU2ODUxMjE1Y2U3YTc2MTc5LzAxN2E5ZWQ1ODhhMTFlZDM4MzcxMTAwM2NmNTBjZjY3NTE5MTQyMGEwN

在train_configs stage2中修改MiniGPT-4 checkpoint路径（已改）

在minigpt4/configs/datasets/cc_sbu/align.yaml修改为数据集路径（已改）

In [ ]:
!apt-get update && apt-get install ffmpeg libsm6 libxext6  -y

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:9 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1,070 kB]
Get:10 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:12 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1,369 kB]
Hit:13 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Get:14 http:

In [ ]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.3/76.3 MB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 653.6/653.6 kB 56.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.3/300.3 kB 35.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 965.4/965.4 kB 65.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 76.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.7/42.7 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.2/280.2 kB 33.1 MB/s et

eval_configs/minigpt4_eval.yaml中的第 11 行设置为待评估模型的checkpoint路径:'/content/drive/MyDrive/instruct_pixel2pixel/checkpoints_finetuned_miniGPT4/output/minigpt4_stage2_finetune/20230630023/checkpoint_4.pth'

In [ ]:
from predict import test
import glob
import json
import os
import shutil

my_test=test()
results=[]

/usr/local/lib/python3.10/dist-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.16) or chardet (5.1.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


Loading VIT


100%|██████████| 1.89G/1.89G [01:28<00:00, 22.9MB/s]


Loading VIT Done
Loading Q-Former


100%|██████████| 413M/413M [00:23<00:00, 18.8MB/s]


Loading Q-Former Done
Loading LLAMA

===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading LLAMA Done
Load 4 training prompts
Prompt Example 
###Human: <Img><ImageHere></Img> Could you describe the contents of this image for me? ###Assistant: 
Load BLIP2-LLM Checkpoint: /content/drive/MyDrive/instruct_pixel2pixel/checkpoints_finetuned_miniGPT4/output/minigpt4_stage2_finetune/20230630023/checkpoint_4.pth


# 将图片转化为caption：

In [ ]:


# Save the results list as a JSON file
output_file = '/content/drive/MyDrive/instruct_pixel2pixel/minigpt4_results.json'

image_folder = "/content/drive/MyDrive/Philly/"
image_paths = glob.glob(image_folder + "*.jpg")  # Update the file extension if necessary
for image_path in image_paths:
    result = my_test.predict(image=image_path, message="use concise language to describe the image in detail. the description should include the following aspects: Weather; road and building; People and activities; lighting and atmosphere.")
    results.append(result)
    print(result)

# Save the results list as a JSON file
with open(output_file, "w") as file:
    json.dump(results, file)


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1219: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


The image shows a road with several cars driving on it. The sky is clear and blue with a few clouds. The road is well maintained and there are no visible signs of traffic congestion. The buildings on either side of the road are modern and well maintained. The atmosphere is bright and sunny, with a few clouds in the sky. There are no people visible in the image. The lighting is bright and well lit, with the sun shining down on the road and buildings. Overall, the image appears to be a typical urban road scene with moderate traffic.
The image shows a street with a building on the left side and a parking lot on the right side. There is a white van driving on the road in front of the building. The sky is clear and blue with a few clouds. The lighting is bright and the atmosphere is calm.
The image shows a city street with several cars parked on the side of the road. The sky is clear and blue, with a few clouds in the distance. The buildings on either side of the street are made of concrete

# 下面不用跑

将下面代码中train.py前的数字改为自己GPU的数量， 这里为1

### 模型训练

In [ ]:
!torchrun --nproc-per-node 1 train.py --cfg-path train_configs/minigpt4_stage2_finetune.yaml

/usr/bin/python3: can't open file '/content/train.py': [Errno 2] No such file or directory
ERROR:torch.distributed.elastic.multiprocessing.api:failed (exitcode: 2) local_rank: 0 (pid: 8319) of binary: /usr/bin/python3
Traceback (most recent call last):
  File "/usr/local/bin/torchrun", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/torch/distributed/elastic/multiprocessing/errors/__init__.py", line 346, in wrapper
    return f(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/torch/distributed/run.py", line 794, in main
    run(args)
  File "/usr/local/lib/python3.10/dist-packages/torch/distributed/run.py", line 785, in run
    elastic_launch(
  File "/usr/local/lib/python3.10/dist-packages/torch/distributed/launcher/api.py", line 134, in __call__
    return launch_agent(self._config, self._entrypoint, list(args))
  File "/usr/local/lib/python3.10/dist-packages/torch/distributed/launcher/api.py", line 250, in launch_agent
    rai

保存output到google drive

In [ ]:

#!cp -r /content/MiniGPT-4/minigpt4/output /content/drive/MyDrive/instruct_pixel2pixel/checkpoints_finetuned_miniGPT4

Mounted at /content/drive


#1.4 模型评估

eval_configs/minigpt4_eval.yaml中的第 11 行设置为待评估模型的checkpoint路径:'/content/drive/MyDrive/instruct_pixel2pixel/checkpoints_finetuned_miniGPT4/output/minigpt4_stage2_finetune/20230630023/checkpoint_4.pth'

In [ ]:
!python demo.py --cfg-path eval_configs/minigpt4_eval.yaml  --gpu-id 0

# 微调前：

eval_configs/minigpt4_eval.yaml中的第 11 行设置为微调前的checkpoint路径

In [ ]:
!python demo.py --cfg-path eval_configs/minigpt4_eval.yaml  --gpu-id 0

/usr/local/lib/python3.10/dist-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.16) or chardet (5.1.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
2023-07-04 20:12:36.339380: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Initializing Chat
Loading VIT
100% 1.89G/1.89G [01:26<00:00, 23.3MB/s]
Loading VIT Done
Loading Q-Former
100% 413M/413M [00:19<00:00, 22.1MB/s]
Loading Q-Former Done
Loading LLAMA

===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
Loading checkpoint shards: 100% 2/2 [00:26<00:00, 13.21s/it]
Loading LLAMA Done
Load 4 training prompts
Prompt Example 
###Human: <Img><ImageHere></Img> Could you describe the contents of this im